# 📷 Image Formation

### Understanding How Cameras See

This notebook covers:
- Pinhole camera model
- Camera calibration
- Lens distortion
- Color spaces
- Illumination models

---


In [ ]:
# Install required libraries
!pip install opencv-python numpy matplotlib scipy -q

import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries installed and imported successfully!")


## 1. Pinhole Camera Model

The pinhole camera model projects 3D world points to 2D image points:

**Projection Equation:** `p = K[R|t]P`

Where:
- **K** (Intrinsic Matrix): Camera internal parameters (focal length, principal point)
- **[R|t]** (Extrinsic Matrix): Camera pose (rotation and translation)
- **P**: 3D world point
- **p**: 2D image point


In [ ]:
# Create a simple pinhole camera projection example

def project_3d_to_2d(P_3d, K, R, t):
    """
    Project 3D points to 2D using pinhole camera model
    
    Args:
        P_3d: 3D points (N x 3)
        K: Intrinsic matrix (3 x 3)
        R: Rotation matrix (3 x 3)
        t: Translation vector (3 x 1)
    
    Returns:
        p_2d: 2D image points (N x 2)
    """
    # Convert to homogeneous coordinates
    P_homogeneous = np.hstack([P_3d, np.ones((P_3d.shape[0], 1))])
    
    # Create extrinsic matrix [R|t]
    Rt = np.hstack([R, t])
    
    # Project: p = K[R|t]P
    p_homogeneous = K @ Rt @ P_homogeneous.T
    p_homogeneous = p_homogeneous.T
    
    # Convert back to Cartesian coordinates
    p_2d = p_homogeneous[:, :2] / p_homogeneous[:, 2:3]
    
    return p_2d

# Example: Define camera parameters
fx, fy = 800, 800  # Focal length in pixels
cx, cy = 320, 240  # Principal point

# Intrinsic matrix K
K = np.array([
    [fx, 0, cx],
    [0, fy, cy],
    [0, 0, 1]
])

# Extrinsic parameters (camera looking at origin)
R = np.eye(3)  # Identity rotation
t = np.array([[0], [0], [5]])  # Camera 5 units back

# Create some 3D points (a simple cube)
cube_points = np.array([
    [0, 0, 0], [1, 0, 0], [1, 1, 0], [0, 1, 0],  # Front face
    [0, 0, -1], [1, 0, -1], [1, 1, -1], [0, 1, -1]  # Back face
])

# Project to 2D
points_2d = project_3d_to_2d(cube_points, K, R, t)

# Visualize
plt.figure(figsize=(10, 6))
plt.scatter(points_2d[:, 0], points_2d[:, 1], s=100, c='red', marker='o')
plt.plot([points_2d[0, 0], points_2d[1, 0]], [points_2d[0, 1], points_2d[1, 1]], 'b-')
plt.plot([points_2d[1, 0], points_2d[2, 0]], [points_2d[1, 1], points_2d[2, 1]], 'b-')
plt.plot([points_2d[2, 0], points_2d[3, 0]], [points_2d[2, 1], points_2d[3, 1]], 'b-')
plt.plot([points_2d[3, 0], points_2d[0, 0]], [points_2d[3, 1], points_2d[0, 1]], 'b-')
plt.gca().invert_yaxis()
plt.title('Pinhole Camera Projection: 3D Cube → 2D Image')
plt.xlabel('u (pixels)')
plt.ylabel('v (pixels)')
plt.grid(True, alpha=0.3)
plt.show()

print(f"✅ Projected {len(cube_points)} 3D points to 2D")
print(f"Image size: 640x480 pixels")


## 2. Camera Calibration

Camera calibration estimates the intrinsic parameters (K) and distortion coefficients.

**Zhang's Method:**
1. Capture multiple images of a checkerboard
2. Detect corner points
3. Solve for K using homography constraints
4. Refine with non-linear optimization


In [ ]:
# Camera Calibration Example
# Note: This requires actual checkerboard images
# Here we'll demonstrate the concept with synthetic data

# Define checkerboard pattern
checkerboard_size = (9, 6)  # Internal corners
square_size = 1.0  # Size of each square in real units

# Generate 3D object points (real-world coordinates)
objp = np.zeros((checkerboard_size[0] * checkerboard_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:checkerboard_size[0], 0:checkerboard_size[1]].T.reshape(-1, 2)
objp *= square_size

# Simulate camera calibration process
print("📐 Camera Calibration Process:")
print("=" * 50)
print("1. Capture multiple images of checkerboard from different angles")
print("2. Detect corner points in each image")
print("3. Match 3D object points to 2D image points")
print("4. Solve for camera matrix K and distortion coefficients")
print("5. Refine using Levenberg-Marquardt optimization")
print()

# Example calibration result (typical values)
K_example = np.array([
    [800, 0, 320],
    [0, 800, 240],
    [0, 0, 1]
])

dist_coeffs_example = np.array([-0.1, 0.05, 0, 0, 0])  # k1, k2, p1, p2, k3

print("Example Calibration Results:")
print(f"Intrinsic Matrix K:\n{K_example}")
print(f"\nDistortion Coefficients: {dist_coeffs_example}")
print(f"  k1, k2: Radial distortion")
print(f"  p1, p2: Tangential distortion")
print(f"  k3: Additional radial term")

# Visualize distortion effect
def apply_distortion(x, y, k1, k2, p1, p2):
    """Apply radial and tangential distortion"""
    r2 = x**2 + y**2
    x_distorted = x * (1 + k1*r2 + k2*r2*r2) + 2*p1*x*y + p2*(r2 + 2*x**2)
    y_distorted = y * (1 + k1*r2 + k2*r2*r2) + p1*(r2 + 2*y**2) + 2*p2*x*y
    return x_distorted, y_distorted

# Create grid of points
x = np.linspace(-1, 1, 20)
y = np.linspace(-1, 1, 20)
X, Y = np.meshgrid(x, y)

# Apply distortion
X_dist, Y_dist = apply_distortion(X, Y, dist_coeffs_example[0], 
                                   dist_coeffs_example[1],
                                   dist_coeffs_example[2],
                                   dist_coeffs_example[3])

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].plot(X, Y, 'b-', alpha=0.3)
axes[0].plot(X.T, Y.T, 'b-', alpha=0.3)
axes[0].set_title('Original Grid (No Distortion)')
axes[0].set_aspect('equal')
axes[0].grid(True, alpha=0.3)

axes[1].plot(X_dist, Y_dist, 'r-', alpha=0.3)
axes[1].plot(X_dist.T, Y_dist.T, 'r-', alpha=0.3)
axes[1].set_title('Distorted Grid (Barrel Distortion)')
axes[1].set_aspect('equal')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ Distortion visualization complete!")


In [ ]:
# Color Space Conversions

# Create a sample image with different colors
height, width = 200, 400
img_rgb = np.zeros((height, width, 3), dtype=np.uint8)

# Create color patches
img_rgb[:100, :100] = [255, 0, 0]      # Red
img_rgb[:100, 100:200] = [0, 255, 0]   # Green
img_rgb[:100, 200:300] = [0, 0, 255]   # Blue
img_rgb[:100, 300:] = [255, 255, 0]    # Yellow
img_rgb[100:, :] = [128, 128, 128]     # Gray

# Convert to different color spaces
img_hsv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV)
img_lab = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2LAB)
img_ycbcr = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2YCR_CB)

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

axes[0, 0].imshow(img_rgb)
axes[0, 0].set_title('RGB Color Space')
axes[0, 0].axis('off')

axes[0, 1].imshow(img_hsv)
axes[0, 1].set_title('HSV Color Space')
axes[0, 1].axis('off')

axes[1, 0].imshow(img_lab)
axes[1, 0].set_title('LAB Color Space')
axes[1, 0].axis('off')

axes[1, 1].imshow(img_ycbcr)
axes[1, 1].set_title('YCbCr Color Space')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

# Show individual channels
fig, axes = plt.subplots(3, 3, figsize=(15, 12))

# RGB channels
axes[0, 0].imshow(img_rgb[:, :, 0], cmap='Reds')
axes[0, 0].set_title('RGB - Red Channel')
axes[0, 1].imshow(img_rgb[:, :, 1], cmap='Greens')
axes[0, 1].set_title('RGB - Green Channel')
axes[0, 2].imshow(img_rgb[:, :, 2], cmap='Blues')
axes[0, 2].set_title('RGB - Blue Channel')

# HSV channels
axes[1, 0].imshow(img_hsv[:, :, 0], cmap='hsv')
axes[1, 0].set_title('HSV - Hue')
axes[1, 1].imshow(img_hsv[:, :, 1], cmap='gray')
axes[1, 1].set_title('HSV - Saturation')
axes[1, 2].imshow(img_hsv[:, :, 2], cmap='gray')
axes[1, 2].set_title('HSV - Value')

# LAB channels
axes[2, 0].imshow(img_lab[:, :, 0], cmap='gray')
axes[2, 0].set_title('LAB - L (Lightness)')
axes[2, 1].imshow(img_lab[:, :, 1], cmap='RdYlGn')
axes[2, 1].set_title('LAB - A (Green-Red)')
axes[2, 2].imshow(img_lab[:, :, 2], cmap='RdYlBu')
axes[2, 2].set_title('LAB - B (Blue-Yellow)')

for ax in axes.flat:
    ax.axis('off')

plt.tight_layout()
plt.show()

print("✅ Color space conversions demonstrated!")
print("\nKey Insights:")
print("- HSV: Hue separates color from brightness (useful for color-based segmentation)")
print("- LAB: Perceptually uniform (small changes = similar perceived difference)")
print("- YCbCr: Separates luminance (Y) from chrominance (Cb, Cr) - used in compression")


## 4. Gamma Correction

Displays have non-linear response. Gamma correction compensates for this:

**Encoding:** `V_out = V_in^(1/γ)` where γ ≈ 2.2  
**Display:** `V_display = V_encoded^γ`  
**Result:** Perceived linear response


In [ ]:
# Gamma Correction Demonstration

def apply_gamma(image, gamma):
    """Apply gamma correction"""
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 
                      for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

# Create linear gradient
linear_gradient = np.linspace(0, 255, 256).astype(np.uint8)
linear_gradient = np.tile(linear_gradient, (100, 1))

# Apply gamma correction (encoding)
gamma = 2.2
gamma_corrected = apply_gamma(linear_gradient, gamma)

# Simulate display (decoding)
display_output = apply_gamma(gamma_corrected, 1/gamma)

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].imshow(linear_gradient, cmap='gray', aspect='auto')
axes[0].set_title('Original Linear Gradient')
axes[0].set_xlabel('Intensity')
axes[0].set_ylabel('Row')

axes[1].imshow(gamma_corrected, cmap='gray', aspect='auto')
axes[1].set_title(f'After Gamma Encoding (γ={gamma})')
axes[1].set_xlabel('Encoded Intensity')

axes[2].imshow(display_output, cmap='gray', aspect='auto')
axes[2].set_title('After Display Decoding (Perceived Linear)')
axes[2].set_xlabel('Perceived Intensity')

for ax in axes:
    ax.set_yticks([])

plt.tight_layout()
plt.show()

# Plot intensity curves
x = np.linspace(0, 1, 256)
y_linear = x
y_encoded = x ** (1/gamma)
y_decoded = y_encoded ** gamma

plt.figure(figsize=(10, 6))
plt.plot(x, y_linear, 'b-', label='Original Linear', linewidth=2)
plt.plot(x, y_encoded, 'r--', label=f'Encoded (γ={gamma})', linewidth=2)
plt.plot(x, y_decoded, 'g:', label='Decoded (Display)', linewidth=2)
plt.xlabel('Input Intensity')
plt.ylabel('Output Intensity')
plt.title('Gamma Correction Curve')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("✅ Gamma correction demonstrated!")
print(f"\nStandard: sRGB uses γ = {gamma}")
print("Purpose: Compensate for non-linear display response")


## 5. Illumination Models

### Lambertian Model (Diffuse Reflection)
`I = kd × (N·L)`

Where:
- **kd**: Diffuse reflectance coefficient
- **N**: Surface normal
- **L**: Light direction

### Phong Model (Adds Specular)
`I = ka + kd(N·L) + ks(R·V)ⁿ`

Where:
- **ka**: Ambient term
- **ks**: Specular coefficient
- **R**: Reflection vector
- **V**: View direction
- **n**: Shininess exponent


In [ ]:
# Illumination Models Visualization

def lambertian_shading(normal, light_dir, kd):
    """Lambertian (diffuse) shading"""
    cos_angle = np.clip(np.dot(normal, light_dir), 0, 1)
    return kd * cos_angle

def phong_shading(normal, light_dir, view_dir, ka, kd, ks, n):
    """Phong shading model"""
    # Ambient
    ambient = ka
    
    # Diffuse (Lambertian)
    cos_angle = np.clip(np.dot(normal, light_dir), 0, 1)
    diffuse = kd * cos_angle
    
    # Specular
    reflect_dir = 2 * np.dot(normal, light_dir) * normal - light_dir
    reflect_dir = reflect_dir / np.linalg.norm(reflect_dir)
    specular = ks * (np.clip(np.dot(reflect_dir, view_dir), 0, 1) ** n)
    
    return ambient + diffuse + specular

# Create a sphere surface
size = 200
x = np.linspace(-1, 1, size)
y = np.linspace(-1, 1, size)
X, Y = np.meshgrid(x, y)
Z = np.sqrt(np.maximum(1 - X**2 - Y**2, 0))

# Surface normals
normals = np.zeros((size, size, 3))
normals[:, :, 0] = X
normals[:, :, 1] = Y
normals[:, :, 2] = Z
normals = normals / (np.linalg.norm(normals, axis=2, keepdims=True) + 1e-8)

# Light and view directions
light_dir = np.array([0.5, 0.5, 1])
light_dir = light_dir / np.linalg.norm(light_dir)
view_dir = np.array([0, 0, 1])

# Shading parameters
ka, kd, ks = 0.1, 0.7, 0.2
n = 32

# Compute shading
lambertian_img = np.zeros((size, size))
phong_img = np.zeros((size, size))

for i in range(size):
    for j in range(size):
        if Z[i, j] > 0:
            normal = normals[i, j]
            lambertian_img[i, j] = lambertian_shading(normal, light_dir, kd)
            phong_img[i, j] = phong_shading(normal, light_dir, view_dir, ka, kd, ks, n)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

im1 = axes[0].imshow(lambertian_img, cmap='gray', vmin=0, vmax=1)
axes[0].set_title('Lambertian Shading (Diffuse Only)')
axes[0].axis('off')
plt.colorbar(im1, ax=axes[0])

im2 = axes[1].imshow(phong_img, cmap='gray', vmin=0, vmax=1)
axes[1].set_title('Phong Shading (Ambient + Diffuse + Specular)')
axes[1].axis('off')
plt.colorbar(im2, ax=axes[1])

plt.tight_layout()
plt.show()

print("✅ Illumination models demonstrated!")
print("\nKey Differences:")
print("- Lambertian: Smooth, matte appearance (no highlights)")
print("- Phong: Adds specular highlights for shiny surfaces")


## 📝 Summary

In this notebook, we covered:

1. **Pinhole Camera Model**: 3D → 2D projection using `p = K[R|t]P`
2. **Camera Calibration**: Estimating intrinsic parameters and distortion
3. **Color Spaces**: RGB, HSV, LAB, YCbCr and their applications
4. **Gamma Correction**: Compensating for non-linear display response
5. **Illumination Models**: Lambertian and Phong shading

### Key Takeaways:
- Understanding image formation is crucial for computer vision
- Camera calibration enables accurate 3D reconstruction
- Color space choice depends on the task
- Gamma correction ensures consistent appearance across displays
- Illumination models help understand how light interacts with surfaces

---

**Next Steps:**
- Try camera calibration with real checkerboard images
- Experiment with color-based segmentation using HSV
- Implement your own shading model
